In [198]:
import numpy as np;
import pandas as pd;
import datetime as dt;
import plotly.graph_objects as go;
import plotly.express as px; 
import matplotlib as mp;

In [199]:
df_Amazon = pd.read_csv('Amazon.csv', sep=';')
df_Amazon

,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
0,ORD0001,14/03/2025,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,ORD0002,20/03/2025,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,ORD0003,15/02/2025,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,ORD0004,19/02/2025,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,ORD0005,10/03/2025,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending
...,...,...,...,...,...,...,...,...,...,...,...
245,ORD0246,17/03/2025,T-Shirt,Clothing,20,2,40,Daniel Harris,Miami,Debit Card,Cancelled
246,ORD0247,30/03/2025,Jeans,Clothing,40,1,40,Sophia Miller,Dallas,Debit Card,Cancelled
247,ORD0248,05/03/2025,T-Shirt,Clothing,20,2,40,Chris White,Denver,Debit Card,Cancelled
248,ORD0249,08/03/2025,Smartwatch,Electronics,150,3,450,Emily Johnson,New York,Debit Card,Cancelled


In [200]:
df_Amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Order ID           250 non-null    object
 1   Date               250 non-null    object
 2   Product            250 non-null    object
 3   Category           250 non-null    object
 4   Price              250 non-null    int64 
 5   Quantity           250 non-null    int64 
 6   Total Sales        250 non-null    int64 
 7   Customer Name      250 non-null    object
 8   Customer Location  250 non-null    object
 9   Payment Method     250 non-null    object
 10  Status             250 non-null    object
dtypes: int64(3), object(8)
memory usage: 21.6+ KB


In [201]:
#1.10 Produtos mais vendidos
quantidades = df_Amazon.groupby('Product')['Quantity'].sum()
percentuais = (quantidades / quantidades.sum()).sort_values(ascending=False).head(10)

fig = px.bar(percentuais, 
             x=percentuais.values, 
             y=percentuais.index, 
             orientation='h',
             title='Top 10 Produtos Mais Vendidos',
             labels={'x': 'Percentual', 'index': 'Produto'},
             text_auto='.2%')
fig.update_layout(yaxis_title=None, xaxis_title='Percentual do Total Vendido')
fig.show()


In [202]:
#2 Quais clientes mais 

vendas_cliente = df_Amazon.groupby('Customer Name')['Total Sales'].sum()
percentuais_cliente = (vendas_cliente / vendas_cliente.sum()).sort_values(ascending=False).head(10)

fig = px.bar(percentuais_cliente, 
             x=percentuais_cliente.values, 
             y=percentuais_cliente.index, 
             orientation='h',
             title='Top 10 Clientes por Valor de Compra',
             labels={'x': 'Percentual', 'index': 'Cliente'},
             text_auto='.2%')
fig.update_layout(yaxis_title=None, xaxis_title='Percentual do Total de Vendas')
fig.show()




In [203]:
#3 Qual a localidade que mais compra?
vendas_local = df_Amazon.groupby('Customer Location')['Total Sales'].sum()
percentuais_local = (vendas_local / vendas_local.sum()).sort_values(ascending=False).head(10)

fig = px.bar(percentuais_local, 
             x=percentuais_local.values, 
             y=percentuais_local.index, 
             orientation='h',
             title='Top 10 Localidades por Vendas',
             labels={'x': 'Percentual', 'index': 'Localidade'},
             text_auto='.2%')
fig.update_layout(yaxis_title=None, xaxis_title='Percentual do Total de Vendas')
fig.show()



In [204]:
pagamentos=df_Amazon['Payment Method'].value_counts()
fig=px.pie(names=pagamentos.index,
           values=pagamentos.values,
           title='Distribuição dos Métodos de Pagamento')
fig.show()

In [205]:
status = df_Amazon['Status'].value_counts(normalize=True)

fig = px.bar(status, 
             x=status.index, 
             y=status.values, 
             title='Status dos Pedidos',
             labels={'x': 'Status', 'y': 'Percentual'},
             text_auto='.2%')
fig.show()


In [206]:
vendas_categoria = df_Amazon.groupby('Category')['Total Sales'].sum()
percentuais_categoria = (vendas_categoria / vendas_categoria.sum()).sort_values(ascending=False)

fig = px.bar(percentuais_categoria, 
             x=percentuais_categoria.index, 
             y=percentuais_categoria.values, 
             title='Vendas por Categoria',
             labels={'x': 'Categoria', 'y': 'Percentual'},
             text_auto='.2%')
fig.show()


🛒 1. Taxa de cancelamento por categoria ou produto
➤ Objetivo: Descobrir onde há maior perda de vendas.

In [207]:


# Filtrar pedidos cancelados
cancelamentos = df_Amazon[df_Amazon['Status'] == 'Cancelled']

# Calcular taxa de cancelamento por categoria
taxa_cancelamento = cancelamentos.groupby('Category')['Order ID'].count() / df_Amazon.groupby('Category')['Order ID'].count()

# Converter para DataFrame para visualização
taxa_cancelamento_df = taxa_cancelamento.reset_index()
taxa_cancelamento_df.columns = ['Categoria', 'Taxa de Cancelamento']
taxa_cancelamento_df = taxa_cancelamento_df.sort_values(by='Taxa de Cancelamento', ascending=False)

# Criar gráfico
fig = px.bar(taxa_cancelamento_df, 
             x='Categoria', 
             y='Taxa de Cancelamento',
             title='Taxa de Cancelamento por Categoria',
             labels={'Categoria': 'Categoria', 'Taxa de Cancelamento': 'Taxa (%)'},
             text=taxa_cancelamento_df['Taxa de Cancelamento'].apply(lambda x: f'{x:.2%}'))

fig.update_traces(textposition='outside')
fig.update_layout(yaxis_tickformat='.0%', xaxis_tickangle=-45)

fig.show()


📉 2. Produtos com maior taxa de pendência ou cancelamento
➤ Objetivo: Identificar gargalos em produtos específicos.

In [208]:


# Agrupamento e cálculo das porcentagens
status_por_produto = df_Amazon.groupby(['Product', 'Status'])['Order ID'].count().unstack().fillna(0)
status_por_produto['% Cancelled'] = status_por_produto['Cancelled'] / status_por_produto.sum(axis=1)
status_por_produto['% Pending'] = status_por_produto['Pending'] / status_por_produto.sum(axis=1)

# Seleciona os 10 produtos com maior % Cancelado
top_10_cancelled = status_por_produto[['% Cancelled', '% Pending']].sort_values(by='% Cancelled', ascending=False).head(10).reset_index()

# Faz o melt, renomeando colunas corretamente
df_melted = top_10_cancelled.melt(id_vars='Product', value_vars=['% Cancelled', '% Pending'], var_name='Status', value_name='value')

# Cria o gráfico com o nome da coluna correto
fig = px.bar(
    df_melted,
    x='value',
    y='Product',
    color='Status',
    orientation='h',
    title='Top 10 Produtos com Maior % de Cancelamentos e Pedidos Pendentes',
    labels={'value': 'Percentual', 'Product': 'Produto', 'Status': 'Status'}
)

fig.update_layout(barmode='group', xaxis_tickformat='.0%')
fig.show()



👟 3. Desempenho por categoria com relação a quantidade e faturamento
➤ Objetivo: Detectar categorias com baixo volume, mas possível valor estratégico.

In [209]:
agrup_cat = df_Amazon.groupby('Category')[['Quantity', 'Total Sales']].sum()
agrup_cat['Ticket Médio'] = agrup_cat['Total Sales'] / agrup_cat['Quantity']
agrup_cat.sort_values(by='Ticket Médio', ascending=False)


,Quantity,Total Sales,Ticket Médio
Category,,,
Home Appliances,110,105000,954.545455
Electronics,348,129950,373.419540
Footwear,72,4320,60.000000
Clothing,115,3540,30.782609
Books,69,1035,15.000000


🗺️ 4. Análise geográfica interna
➤ Objetivo: Investigar se a performance de cidades está relacionada a produtos, categoria ou status.

In [210]:
df_Amazon.groupby(['Customer Location', 'Category'])['Total Sales'].sum().unstack().fillna(0).head(10)


Category,Books,Clothing,Electronics,Footwear,Home Appliances
Customer Location,,,,,
Boston,120.0,320.0,12950.0,180.0,12600.0
Chicago,210.0,260.0,12900.0,240.0,7200.0
Dallas,75.0,940.0,6450.0,480.0,19200.0
Denver,75.0,400.0,13950.0,360.0,15000.0
Houston,30.0,80.0,17900.0,780.0,9600.0
Los Angeles,30.0,80.0,11050.0,660.0,6000.0
Miami,120.0,100.0,20500.0,780.0,10200.0
New York,30.0,800.0,10550.0,360.0,7200.0
San Francisco,255.0,160.0,12900.0,480.0,2400.0


💳 5. Relacionamento entre método de pagamento e status do pedido
➤ Objetivo: Identificar se há correlação entre método de pagamento e pedidos cancelados ou pendentes.

In [211]:
# Mesmo cálculo sem multiplicar por 100
pagamento_status['% Cancelled'] = pagamento_status['Cancelled'] / pagamento_status.sum(axis=1)
pagamento_status['% Pending'] = pagamento_status['Pending'] / pagamento_status.sum(axis=1)

# Formata a visualização para porcentagem com 1 casa decimal
pagamento_status[['% Cancelled', '% Pending']]\
    .sort_values(by='% Cancelled', ascending=False)\
    .style.format("{:.1%}")


Status,% Cancelled,% Pending
Payment Method,,
Gift Card,42.0%,42.0%
Debit Card,37.2%,35.4%
Credit Card,29.3%,38.4%
PayPal,26.4%,23.1%
Amazon Pay,16.9%,31.3%


📈 6. Aprofundar a análise temporal
➤ Objetivo: Identificar tendências de venda e variações de comportamento.

In [212]:

# Converter a coluna 'Date' para datetime
df_Amazon['Date'] = pd.to_datetime(df_Amazon['Date'])

# Criar a coluna com ano e mês
df_Amazon['AnoMes'] = df_Amazon['Date'].dt.to_period('M').astype(str)

# Agrupar vendas por mês e categoria
vendas_categoria = df_Amazon.groupby(['AnoMes', 'Category'])['Total Sales'].sum().reset_index()

# Criar gráfico interativo
fig = px.line(vendas_categoria,
              x='AnoMes',
              y='Total Sales',
              color='Category',
              title='Vendas por Categoria ao Longo do Tempo')

# Personalizar visual
fig.update_traces(mode='lines+markers')
fig.update_layout(xaxis_title='Ano-Mês', yaxis_title='Total de Vendas (R$)',
                  legend_title='Categoria')

fig.show()


C:\Users\jehgo\AppData\Local\Temp\ipykernel_24124\1451420527.py:2: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

